In [ ]:
from geonate import common, raster, plot
import folium
import numpy as np
import rasterio

import folium
import rasterio
import numpy as np
from folium.raster_layers import ImageOverlay
from rasterio.plot import reshape_as_image
import pyproj

from typing import AnyStr, Dict, Optional

In [23]:
def plot_raster(input, layername: Optional[AnyStr]=None, rgb: Optional[list]=None, stretch: Optional[AnyStr]='linear', brightness: Optional[float]=None, contrast: Optional[float]=None, opacity: Optional[float]=1, zoom: Optional[float]=5, basemap: Optional[AnyStr]='OSM', output: Optional[AnyStr]= None):
    """
    Plots a basemap with an overlay of raster data.

    Args:
        input (DatasetReader): The input raster dataset.
        layername (Anstr, optional): Layer name of image.
        rgb (list, optional): List of RGB bands to visualize. Defaults to None.
        stretch (AnyStr, optional): Stretch method for the image ('linear', 'hist', 'custom'). Defaults to 'linear'.
        brightness (float, optional): Brightness value for custom stretch. Defaults to None.
        contrast (float, optional): Contrast value for custom stretch. Defaults to None.
        opacity (float, optional): Opacity of the image overlay. Defaults to 1.
        zoom (float, optional): Initial zoom level of the map. Defaults to 5.
        basemap (AnyStr, optional): Basemap type ('OSM', 'CartoDB Positron', 'CartoDB Dark Matter', 'OpenTopoMap', 'Esri Satellite', 'Esri Street Map', 'Esri Topo', 'Esri Canvas'). Defaults to 'OSM'.
        output (AnyStr, optional): File path to write out html file to local directory. Defaults to None.

    Returns:
        folium.Map: A folium map object with the raster data overlay.

    """
    import folium
    from folium.raster_layers import ImageOverlay
    import rasterio
    import numpy as np
    import geonate.common
    import geonate.raster
    
    # Take basemap
    if basemap.lower() == 'openstreetmap' or basemap.lower() == 'osm' or basemap.lower() == 'open street map':
        basemap_name = 'OpenStreetMap'
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom, tiles=basemap_name)

    elif basemap.lower() == 'cartodbpositron' or basemap.lower() == 'cartodb positron' or basemap.lower() == 'light' :
        basemap_name = 'Cartodb Positron'
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom, tiles=basemap_name)

    elif basemap.lower() == 'cartodbdarkmatter' or basemap.lower() == 'cartodb dark matter' or basemap.lower() == 'cartodb dark' or basemap.lower() == 'dark':
        basemap_name = 'Cartodb dark_matter'
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom, tiles=basemap_name)
    
    elif basemap.lower() == 'opentopomap' or basemap.lower() == 'opentopo' or basemap.lower() == 'topo':
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom)
        folium.TileLayer(
            tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
            attr='&copy; Topo Map',
            name='Open Topo Map'
        ).add_to(m)

    elif basemap.lower() == 'esri satellite' or basemap.lower() == 'esrisatellite' or basemap.lower() == 'satellite':
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom)
        folium.TileLayer(
            tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='&copy; Esri',
            name='Esri Satellite'
        ).add_to(m)

    elif basemap.lower() == 'esri street' or basemap.lower() == 'esristreet' or basemap.lower() == 'streetmap' or basemap.lower() == 'street map':
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom)
        folium.TileLayer(
            tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}',
            attr='&copy; Esri',
            name='Esri Street Map'
        ).add_to(m)

    elif basemap.lower() == 'esri topo' or basemap.lower() == 'esritopo':
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom)
        folium.TileLayer(
            tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
            attr='&copy; Esri',
            name='Esri Topo Map'
        ).add_to(m)

    elif basemap.lower() == 'esri canvas' or basemap.lower() == 'esricanvas' or basemap.lower() == 'canvas':
        m = folium.Map(location=[lat_center, lon_center], zoom_start= zoom)
        folium.TileLayer(
            tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
            attr='&copy; Esri',
            name='Esri Canvas Gray'
        ).add_to(m)

    else:
        raise ValueError("Basemap is not supported, please select one of these maps ('OSM', 'CartoDB Positron', 'CartoDB Dark Matter', 'OpenTopoMap', 'Esri Satellite', 'Esri Street Map', 'Esri Topo', 'Esri Canvas')")

    # Add image to basemap 

    ### Check input data is raster or not and extract information
    for input_rast in input:  
        if isinstance(input_rast, rasterio.DatasetReader):
            # Convert image to lat/long if input is not in lat/long system
            crs = input_rast.crs.to_string()
            if crs == "EPSG:4326":
                input_converted = input_rast
            else:
                resolution_degree = common.meter2degree(input_rast.res[0])
                reprojected, meta = raster.reproject(input_rast, 'EPSG:4326', res=resolution_degree)
                input_converted = common.array2raster(reprojected, meta)

            # Extract data from image to visualize
            if (input_rast.count <= 2):
                print('Input image/data has less than 2 bands, it will load the first band only')
                dataset = input_rast.read(1)
                imgData = dataset[:, :, np.newaxis]
            elif (input_rast.count >= 3):
                if rgb is None: 
                    raise ValueError('Input is multiple band image, please provide rgb bands to visualize [3,2,1]')
                else:
                    dataset = input_rast.read(rgb)
                    imgData = np.transpose(dataset, (1, 2, 0)) # Transpose from raster dims (bands, width, height) to image dims (width, height, bands)
        else:
            raise ValueError("Input data is not supported. It must be raster image")

        # Check stretch method
        if stretch is None:
            data = imgData

        elif stretch.lower() == 'linear':
            data = np.clip((imgData  - imgData.min()) / (imgData.max() - imgData.min()) * 255, 0, 255).astype(np.uint8) # linear stretching based on min max values

        elif stretch.lower() == 'hist' or stretch.lower() == 'histogram':
            from skimage import exposure
            data = exposure.equalize_hist(imgData)  # This returns a floating point image with values between 0 and 1
            data = (data * 255).astype(np.uint8)  # Convert back to 8-bit image for display

        elif stretch.lower() == 'custom':
            if (contrast is None) or (brightness is None):
                raise ValueError("contrast and brightness must be given for custom stretching method")
            else:
                data = np.clip((imgData * contrast + brightness), 0, 255).astype(np.uint8)

        else: 
            raise ValueError("Stretch method is not supported ('linear', 'hist', 'custom')")
        
        # Get Bounds values
        left, bottom, right, top = common.getBounds(input_converted)
        lat_center = (top + bottom) / 2
        lon_center = (left + right)/ 2
        bounds = [[bottom, left], [top, right]]
        
        # Create overlay image
        if layername is not None:
            image_overlay = ImageOverlay(image= data, bounds= bounds, opacity= opacity, name=layername)
        else:
            image_overlay = ImageOverlay(image= data, bounds= bounds, opacity= opacity, name='Layer')

        # Add the image overlay to the map
        image_overlay.add_to(m)


    folium.LayerControl().add_to(m)

    # Save map
    if output is not None:
        m.save(output)
    else:
        pass

    return m  

In [9]:
p1 = './Sample_data/landsat_multi/Scene/landsat_img_00.tif'
p2 = './Sample_data/landsat_multi/Scene/landsat_img_01.tif'
from geonate import raster
import rasterio
img1 = raster.rast(p1)
img2 = raster.rast(p2)

imgs = [img1, img2]


def get_combined_extent_from_opened(datasets):
    """
    Returns the combined extent of multiple *already opened* rasterio datasets.

    Args:
        datasets: A list of rasterio Dataset objects (already opened).

    Returns:
        A tuple representing the combined extent (west, south, east, north), or None if there's an issue.
        Raises a ValueError if the input list is empty or contains non-rasterio Dataset objects.
    """

    if not datasets:
        raise ValueError("Input list of datasets cannot be empty.")

    if not all(isinstance(dataset, rasterio.DatasetReader) for dataset in datasets):
        raise ValueError("All elements in the list must be rasterio Dataset objects.")


    try:
        # Initialize with the bounds of the first dataset
        west = datasets[0].bounds.left
        south = datasets[0].bounds.bottom
        east = datasets[0].bounds.right
        north = datasets[0].bounds.top

        # Iterate through the rest and update
        for dataset in datasets[1:]:
            west = min(west, dataset.bounds.left)
            south = min(south, dataset.bounds.bottom)
            east = max(east, dataset.bounds.right)
            north = max(north, dataset.bounds.top)

        return (west, south, east, north)

    except Exception as e:  # Catch other potential errors
        print(f"An error occurred: {e}")
        return None


# Example usage (assuming you've already opened the datasets):
datasets = imgs
a, b, c, d = get_combined_extent_from_opened(datasets)

a, b, c, d


(599655.0, 1128975.0, 626055.0, 1138185.0)

In [26]:
common.extent(imgs)

: 

In [ ]:
m.add_to(m1)

In [ ]:
def plot_vect(input: list, name: Optional[list], ):
    import folium
    import geopandas as gpd

    

In [ ]:
shapefile_path = "./Sample_data/shapefile/GCP_polys.shp"

import folium
import geopandas as gpd

# Load shapefiles (replace with your actual paths)
polygon_gdf = gpd.read_file("path/to/polygon_shapefile.shp")
multipoint_gdf = gpd.read_file("path/to/multipoint_shapefile.shp")

# Ensure correct CRS
polygon_gdf = polygon_gdf.to_crs(epsg=4326)
multipoint_gdf = multipoint_gdf.to_crs(epsg=4326)

# Get the centroid for map centering
centroid = polygon_gdf.geometry.centroid.unary_union

# Create base map
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

# Add Polygon Layer
folium.GeoJson(polygon_gdf, name="Polygons", style_function=lambda x: {"color": "red"}).add_to(m)

# Create a FeatureGroup for MultiPoints
multipoint_layer = folium.FeatureGroup(name="MultiPoints")

# Iterate through MultiPoint geometries and add them as individual markers
for _, row in multipoint_gdf.iterrows():
    geom = row.geometry
    if geom.geom_type == "MultiPoint":
        for point in geom.geoms:  # Iterate through individual points
            folium.Marker(
                location=[point.y, point.x],
                popup=f"MultiPoint: {point.x}, {point.y}",
                icon=folium.Icon(color="blue", icon="info-sign"),
            ).add_to(multipoint_layer)
    elif geom.geom_type == "Point":  # Handle single Point geometries
        folium.Marker(
            location=[geom.y, geom.x],
            popup=f"Point: {geom.x}, {geom.y}",
            icon=folium.Icon(color="green", icon="info-sign"),
        ).add_to(multipoint_layer)

# Add MultiPoint layer to map
multipoint_layer.add_to(m)

# Add Layer Control
folium.LayerControl().add_to(m)

# Save map
m.save("multi_layer_map.html")
m  # If using Jupyter Notebook



In [ ]:
import folium
import geopandas as gpd
from folium.plugins import Fullscreen

# Load the shapefile (replace 'your_shapefile.shp' with your file path)
shapefile_path = "./Sample_data/shapefile/GCP_polys.shp"
gdf = gpd.read_file(shapefile_path)

# Check the CRS and reproject if necessary
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Get the centroid of the geometries to center the map
centroid = gdf.geometry.unary_union.centroid
map_center = [centroid.y, centroid.x]

# Create a Folium map
m = folium.Map(location=map_center, zoom_start=10, tiles="cartodb positron")
Fullscreen().add_to(m)  # Add fullscreen option

# Function to add polygons to the map
def add_geometries(gdf, map_obj):
    for _, row in gdf.iterrows():
        geo_json = folium.GeoJson(
            row.geometry,
            name=row.get("name", "Polygon"),  # Change "name" to an existing column if available
            style_function=lambda x: {
                "fillColor": "blue",
                "color": "black",
                "weight": 1,
                "fillOpacity": 0.5,
            },
        )
        geo_json.add_to(map_obj)

# Add all geometries to the map
add_geometries(gdf, m)

# Add a layer control
folium.LayerControl().add_to(m)

# Save map to an HTML file
m.save("shapefile_map.html")

# Display map in Jupyter Notebook (if running in a Jupyter environment)
m